# 导入数据

In [1]:
import pandas as pd
app = pd.DataFrame(columns = ['userId','appId'])
dev = pd.DataFrame(columns = ["userId", "device"])
aoi = pd.DataFrame(columns = ["userId", "location"])
for date in range(20170306, 20170320):
    daily_app = pd.read_table("/home/hadoop/sdl/hdfs_data/66/lkd/" + str(date) + "/app/part-00000", 
                              header = None, names = ["userId", "appId"])
    daily_dev = pd.read_table("/home/hadoop/sdl/hdfs_data/66/lkd/" + str(date) + "/dev/part-00000", 
                              header = None, names = ["userId", "device"])
    daily_aoi = pd.read_table("/home/hadoop/sdl/hdfs_data/66/lkd/" + str(date) + "/aoi/part-00000", 
                              header = None, names = ["userId", "location"])
    
    app = app.append(daily_app)
    dev = dev.append(daily_dev)
    aoi = aoi.append(daily_aoi)
app = app.reset_index().drop('index', axis = 1)
dev = dev.reset_index().drop('index', axis = 1)
aoi = aoi.reset_index().drop('index', axis = 1)

# 将device信息转化成ios和Android作为特征

In [6]:
dev['IOS'] = dev.device.apply(lambda x: 1 if 'iPhone' in x or 'iPad' in x or 'iPod' in x else 0)
dev['Android'] = dev.device.apply(lambda x: 0 if 'iPhone' in x or 'iPad' in x or 'iPod' in x not in x else 1)
dev = dev.drop_duplicates(subset = 'userId',keep = 'first',inplace = False)
dev.head()

,userId,device,IOS,Android
0,111,HUAWEI:FRD-AL00,0,1
1,4801,"iPhone8,1",1,0
2,13024,OPPO A33,0,1
3,13491,SM-G9008V,0,1
4,18666,Xiaomi:MI NOTE LTE,0,1


# 处理aoi地理位置信息

In [10]:
aoi_names=[]
aoi={}
for day in range(20170306,20170320):
    file=open("/home/hadoop/sdl/hdfs_data/66/lkd/"+str(day)+"/aoi/part-00000",'r')
    for i in open("/home/hadoop/sdl/hdfs_data/66/lkd/"+str(day)+"/aoi/part-00000",'r'):
        line=file.readline()
        line=line.strip().split('\t')
        if line[0] not in aoi:
            aoi[line[0]]={}
        for x in line[1].split(' '):
            if x in aoi[line[0]]:
                aoi[line[0]][x]+=1
            else:
                aoi[line[0]][x]=1
            if x not in aoi_names:
                aoi_names.append(x)
dev_id=[x for x in aoi]
aoi_data=pd.DataFrame({'userId':dev_id})
for x in aoi_names:
    aoi_data[x]=[aoi[i][x] if x in aoi[i] else 0 for i in dev_id]
print(aoi_data.shape)
aoi_data = aoi_data.drop(columns = ['null'])
aoi_data.head()

(78477, 36)


,userId,景区,医院,小区,工业园区,大型体育馆/体育中心,购物中心,公园,政府机构,学校,...,度假村,陵园,文化宫,滑雪场,科技馆,公司,航站楼,公交枢纽,渡口码头,养殖场
0,91139,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35540,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40135,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,73399,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11542,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 处理app信息

In [13]:
appid_counts={}
app={}
app_days={}
for day in range(20170306, 20170320):
    print(day)
    file = open("/home/hadoop/sdl/hdfs_data/66/lkd/" + str(day) + "/app/part-00000", 'r')
    j=0
    for i in open("/home/hadoop/sdl/hdfs_data/66/lkd/" + str(day) + "/app/part-00000", 'r'):
        line=file.readline()
        line=line.strip().split('\t')
        if line[0] not in app:
            app[line[0]]={}
        if line[0] in app_days:
            app_days[line[0]]['days']+=1
            app_days[line[0]]['app_num']+=len(line[1].split(' '))
        else:
            app_days[line[0]]={'days':1}
            app_days[line[0]]['app_num']=len(line[1].split(' '))
        for x in line[1].split(' '):
            if x in app[line[0]]:
                app[line[0]][x]+=1
            else:
                app[line[0]][x]=1
            if x in appid_counts:
                appid_counts[x]+=1
            else:
                appid_counts[x]=1
                
appids=[x for x in appid_counts]
appid=pd.DataFrame({'appid':appids,'frequency':[appid_counts[x] for x in appids]})
appid=appid.sort_values('frequency',ascending=False)

app_tdid=[x for x in app_days]
app_data=pd.DataFrame({'userId':app_tdid})

    
appid=appid.reset_index()
x=appid['appid']
for x in appid_counts:
    if appid_counts[x]>4250:#按app的使用次数筛选app
        app_data[x]=[app[i][x] if x in app[i] else 0 for i in app_tdid]
print(app_data.shape)
app_data.head(10)

20170306
20170307
20170308
20170309
20170310
20170311
20170312
20170313
20170314
20170315
20170316
20170317
20170318
20170319
(84572, 301)


,userId,23710,82285,23032,87478,6472,60996,446,591,42287,...,85150,86577,22470,34637,80490,40840,56532,69610,81102,70818
0,91139,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35540,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40135,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,110554,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11542,0,0,0,0,0,0,0,0,0,...,0,0,0,10,0,0,0,0,0,0
5,89371,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,121036,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,89373,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,11546,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,11547,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
print(len(dev))
print(len(app_data))
print(len(aoi_data))

84572
84572
78477


# 合并aoi/app/dev得到full_df

In [18]:
aoi_data[['userId']] = aoi_data[['userId']].astype(str)
dev[['userId']] = dev[['userId']].astype(str)
full_df_1 = pd.merge(app_data,aoi_data,on = ['userId'], how ='outer')
full_df = pd.merge(full_df_1,dev, on =['userId'],how = 'outer')
#full_df = full_df.set_index('userId')
print(full_df.shape)
full_df.head()

(84572, 338)


,userId,23710,82285,23032,87478,6472,60996,446,591,42287,...,滑雪场,科技馆,公司,航站楼,公交枢纽,渡口码头,养殖场,device,IOS,Android
0,91139,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,heexon:X18,0,1
1,35540,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,:OPPO R9sk,0,1
2,40135,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SM-G8508S,0,1
3,110554,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"iPhone7,1",1,0
4,11542,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3005,0,1


# 导入种子用户并匹配userId

In [47]:
seed_1 = pd.read_table('/home/hadoop/sdl/hdfs_data/66/lkd/seed/1_visible/part-00000',header = None)
seed_1.head()
seed_1.columns = ['userId']

seed_2 = pd.read_table('/home/hadoop/sdl/hdfs_data/66/lkd/seed/2_visible/part-00000',header = None)
seed_2.head()
seed_2.columns = ['userId']

Id1 = list(set(seed_1['userId']))
Id2 = list(set(seed_2['userId']))

seed_1[['userId']]=seed_1[['userId']].astype(str)
seed1_df = pd.merge(seed_1,full_df,how = 'left')
seed_2[['userId']]=seed_2[['userId']].astype(str)
seed2_df = pd.merge(seed_2,full_df,how = 'left')

In [48]:
seed1_df = seed1_df.set_index('userId')
seed1_df = seed1_df.dropna(axis = 0,how = 'all')
seed2_df = seed2_df.set_index('userId')
seed2_df = seed2_df.dropna(axis = 0,how = 'all')

# 种子用户去重

In [49]:
seed1_df = seed1_df[~seed1_df.index.duplicated()]
seed2_df = seed2_df[~seed2_df.index.duplicated()]

In [51]:
seed1_df = seed1_df.drop(['device'],1)
seed2_df = seed2_df.drop(['device'],1)

# 选择全部数据中除种子用户之外的用户作为负样本

In [78]:
nt = []
for i in range(len(full_df)):
    nt.append(full_df['userId'][i])
    
not_seed1 = set(nt) - set([str(Id) for Id in Id1])
not_seed2 = set(nt) - set([str(Id) for Id in Id2])
not_seed1_df = full_df[full_df.userId.isin(not_seed1)]
not_seed2_df = full_df[full_df.userId.isin(not_seed2)]
not_seed1_df = not_seed1_df.fillna(0)
not_seed2_df = not_seed2_df.fillna(0)
seed1_df = seed1_df.fillna(0)
seed2_df = seed2_df.fillna(0)

In [79]:
not_seed1_df = not_seed1_df.drop(['device'],1)
not_seed2_df = not_seed2_df.drop(['device'],1)

In [72]:
seed1_df = seed1_df.set_index('userId')
seed2_df = seed2_df.set_index('userId')

In [82]:
not_seed1_df = not_seed1_df.set_index('userId')
not_seed2_df = not_seed2_df.set_index('userId')

# 开始训练模型

In [56]:
from __future__ import division, print_function
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [83]:
%%time

data_P = seed2_df
data_U = not_seed2_df
NP = seed2_df.shape[0]
NU = not_seed2_df.shape[0]

T = 100
K = NP
train_label = np.zeros(shape=(NP+K,))
train_label[:NP] = 1.0
n_oob = np.zeros(shape=(NU,))
f_oob = np.zeros(shape=(NU, 2))
for i in range(T):
    # Bootstrap resample
    bootstrap_sample = np.random.choice(np.arange(NU), replace=True, size=K)
    # Positive set + bootstrapped unlabeled set
    data_bootstrap = pd.concat((data_P, data_U.iloc[bootstrap_sample, :]), axis=0)
    # Train model
    #model = DecisionTreeClassifier(max_depth=None, max_features=None, 
    #                               criterion='gini', class_weight='balanced')
    model = XGBClassifier(n_jobs = 4)
    model.fit(data_bootstrap, train_label)
    # Index for the out of the bag (oob) samples
    idx_oob = sorted(set(range(NU)) - set(np.unique(bootstrap_sample)))
    # Transductive learning of oob samples
    f_oob[idx_oob] += model.predict_proba(data_U.iloc[idx_oob])
    n_oob[idx_oob] += 1
    print(i)
predict_proba2 = f_oob[:, 1]/n_oob

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
CPU times: user 3min 15s, sys: 23 s, total: 3min 39s
Wall time: 1min 44s


In [99]:
pred_seed2 = not_seed2_df
pred_seed2['probability'] = predict_proba2
pred_seed2 = pd.DataFrame(pred_seed2['probability'])
pred_seed2.head()

,probability
userId,
91139,0.003883
35540,0.003972
40135,0.002722
110554,0.818867
11542,0.002692
